In [1]:
import scipy
import numpy as np
import pandas as pd
import plotly.plotly as py
# import visplots
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
from sklearn import preprocessing, metrics
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from scipy.stats.distributions import randint
import csv
import numpy as np
from pandas import Series, DataFrame
import plotly.graph_objs as go
from plotly import tools

from nltk import word_tokenize, wordpunct_tokenize
from nltk.tokenize import wordpunct_tokenize
from nltk.probability import FreqDist, DictionaryProbDist, ELEProbDist, sum_logs
from nltk.classify.api import ClassifierI

# getting around the ascii characters
from django.utils.encoding import smart_str, smart_unicode

In [21]:
# Import the clean tweets (text and date, then through machine learning)
twitter_raw = pd.read_csv("C:\\Users\\Student21\\Documents\\Training\\Weather Project\\Final Classify\\Clean_Tweets1.csv", sep=',', delimiter=None)

# Delete tweets containing key words

In [22]:
twitter_cleaned=twitter_raw[twitter_raw['text'].str.lower().str.contains \
                   ("great smog of london|spanish|sex|porn|anal|pov|bbw|milf|sexy|shemale|sexyfishrestaurant|\
                   nude|sluts|super hot blonde|adult video|erotic|18+|dirty fun|killer fog|rihanna")==False]

twitter_cleaned = twitter_cleaned.reset_index(drop=True)

twitter_cleaned['text'].count()

8940

#  Machine Learning to clean tweets

# 1. Create the Maching learning

In [4]:
twlist = []

with open(r"C:\\Users\\Student21\\Documents\\Training\\Weather Project\\trained_weather_NoRT.csv", "r") as t:
    tweets_raw = pd.read_csv(t)

tweets = tweets_raw[['text', 'weather']].values.tolist()

twlist = [tuple(l) for l in tweets] # turn nested list of lists into list of tuples
twtokens = []

In [5]:
for (words, weather) in twlist:
#    words_filtered = [e.lower().decode('utf8') for e in words.split() if len(e) >= 3 and len(e) <= 10] # and <= 10
    words_filtered = [unicode(e.lower(), errors = 'replace') for e in words.split() if len(e) >= 3 and len(e) <= 10] # and <= 10
    twtokens.append((words_filtered, weather))

In [6]:
import nltk
def get_words_in_tweets(tweets):
    all_words = []
    for (words, weather) in tweets:
        all_words.extend(words)
    return all_words

def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    word_features = wordlist.keys()
    return word_features

In [7]:
word_features = get_word_features(get_words_in_tweets(twtokens))

In [8]:
# The Classifier
def extract_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in document_words)
    return features

In [9]:
training_set = nltk.classify.apply_features(extract_features, tweets)

# train the classifier
classifier = nltk.NaiveBayesClassifier.train(training_set)

print classifier.show_most_informative_features(32)

Most Informative Features
    contains(summer���s) = False              no : yes    =      1.0 : 1.0
           contains(66%) = False              no : yes    =      1.0 : 1.0
      contains(forecast) = False              no : yes    =      1.0 : 1.0
     contains(@thedoors) = False              no : yes    =      1.0 : 1.0
      contains(@pc24fym) = False              no : yes    =      1.0 : 1.0
    contains(infinit...) = False              no : yes    =      1.0 : 1.0
       contains(finally) = False              no : yes    =      1.0 : 1.0
        contains(1012mb) = False              no : yes    =      1.0 : 1.0
       contains(provide) = False              no : yes    =      1.0 : 1.0
    contains(@moosawi17) = False              no : yes    =      1.0 : 1.0
      contains(mentally) = False              no : yes    =      1.0 : 1.0
          contains(#now) = False              no : yes    =      1.0 : 1.0
         contains(rainy) = False              no : yes    =      1.0 : 1.0

# 2. Classify London Tweets

In [23]:
df = pd.DataFrame(twitter_cleaned)

tx = df['text']
df['text'] = tx

In [24]:
dffinal = df[['text']]
dffinal['date'] = df['Date']

In [25]:
for i in range(len(df.index)):
    if classifier.classify(extract_features((smart_str(df['text'][i])).split())) == 'yes':
        dffinal['text'][i] = smart_str(df['text'][i])
    else:
        dffinal['text'][i] = None

count = 0
for i in range(len(df.index)):
    if dffinal['text'][i] != None:
        count += 1

count2 = 0
dffinaltrained = pd.DataFrame({'date' : pd.Series(range(count), index=range(count)), 'text' : pd.Series(range(count), index=range(count))})
for i in range(len(df.index)):
    if dffinal['text'][i] != None:
        dffinaltrained['text'][count2] = dffinal['text'][i]
        dffinaltrained['date'][count2] = dffinal['date'][i]
        count2 += 1

C:\Users\Student21\Anaconda2\lib\site-packages\ipykernel\__main__.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [26]:
dffinaltrained['text'].count()

8093

# Define function which will count the weather words

In [52]:
def wordCount(tweet):
    # List of words we are looking at
    weather_words = ['breeze', 'breezy', 'cloudy', 'cold', 'ice', 'icy', 'icey', 'drizzle', 'frost', 'wind', 'mild', 'dew', 
                     'freezing', 'downpour', 'shower', 'rain', 'frost', 'nippy', 'hail', 'temperature', 'gail', 'gust',
                     'sleet', 'heat', 'storm', 'slush', 'fog', 'foggy', 'flood', 'visibility', 'warm', 'warmer',
                     'mist', 'frosty', 'misty', 'chilly', 'thunder', 'lightning', 'snow', 'snowing', 'hot', 'sun', 'sunny',
                     'boiling', 'baltic', 'burn']
    # Create a new dictionnary
    counts = dict()
    for t in tweet:
        if t in weather_words:
            counts[t] = counts.get(t,0) + 1

    return counts

# Get the number of final clean tweets per day

In [53]:
# Check the days and number of tweets per day
dffinaltrained.groupby(['date']).count()

,text
date,
2016-12-14,300
2016-12-15,255
2016-12-16,795
2016-12-19,791
2016-12-20,804
2016-12-21,421
2016-12-22,463
2016-12-23,438
2016-12-24,447


In [54]:
# Create a new dataframe with the date
tw_date = dffinaltrained.groupby(['date']).count()
tw_date.index

Index([u'2016-12-14', u'2016-12-15', u'2016-12-16', u'2016-12-19',
       u'2016-12-20', u'2016-12-21', u'2016-12-22', u'2016-12-23',
       u'2016-12-24', u'2016-12-25', u'2016-12-26', u'2016-12-27',
       u'2016-12-28', u'2016-12-30', u'2016-12-31', u'2017-01-01',
       u'2017-01-02'],
      dtype='object', name=u'date')

In [71]:
tw_date['text'][1]

255

# Going through all tweets then count words and put them in final DataFrame (per day)

In [55]:
# Initialise the final dataframe
tw_final = DataFrame()
tw_final['date'] = tw_date.index

tw_final['Keyword_1'] = None
tw_final['Keyword_2'] = None
tw_final['Keyword_3'] = None
tw_final['Keyword_4'] = None
tw_final['Keyword_5'] = None

In [111]:
for i in range(len(tw_date.index)):
    df = dffinaltrained[(dffinaltrained['date'] == tw_final['date'][i])]
    twitterlist = []
    for t in range(len(df.index)):
        twitterlist.append(dffinaltrained['text'][t].lower().split())

    # Flatten the list so all values in the same list
    tweets_flatten = [j for sublist in twitterlist for j in sublist]

    # Counting the number of key words in the tweets
    d = wordCount(tweets_flatten)
    
    # give a list of the sorted tweets, with most popular coming first
    d2 = sorted(d.items(), key = lambda x: x[1], reverse = True)

    
    tw_final['Keyword_1'][i] = d2[0]#[0]  #keyword 1 is the most popular
    tw_final['Keyword_2'][i] = d2[1]#[0]  #keyword 2 is the second most popular
    tw_final['Keyword_3'][i] = d2[2]#[0]
    tw_final['Keyword_4'][i] = d2[3]#[0]
    tw_final['Keyword_5'][i] = d2[4]#[0]

In [113]:
tw_final

,date,Keyword_1,Keyword_2,Keyword_3,Keyword_4,Keyword_5
0,2016-12-14,"(fog, 35)","(hot, 33)","(cold, 33)","(sun, 32)","(warm, 17)"
1,2016-12-15,"(fog, 31)","(hot, 29)","(sun, 28)","(cold, 28)","(warm, 16)"
2,2016-12-16,"(hot, 74)","(snow, 66)","(fog, 61)","(cold, 60)","(sun, 60)"
3,2016-12-19,"(hot, 74)","(snow, 66)","(fog, 61)","(cold, 60)","(sun, 60)"
4,2016-12-20,"(hot, 76)","(snow, 66)","(cold, 63)","(fog, 61)","(sun, 60)"
5,2016-12-21,"(hot, 47)","(sun, 42)","(fog, 42)","(cold, 41)","(warm, 28)"
6,2016-12-22,"(hot, 55)","(fog, 49)","(sun, 45)","(cold, 43)","(warm, 32)"
7,2016-12-23,"(hot, 50)","(fog, 45)","(sun, 44)","(cold, 41)","(warm, 29)"
8,2016-12-24,"(hot, 53)","(fog, 46)","(sun, 44)","(cold, 42)","(warm, 30)"
9,2016-12-25,"(hot, 55)","(fog, 48)","(sun, 45)","(cold, 43)","(warm, 31)"


# Rahul's Join

In [114]:
# Unpivots tw_final
tw_final2 = pd.melt(tw_final, id_vars = ['date'])

In [115]:
tw_final2.head()

,date,variable,value
0,2016-12-14,Keyword_1,"(fog, 35)"
1,2016-12-15,Keyword_1,"(fog, 31)"
2,2016-12-16,Keyword_1,"(hot, 74)"
3,2016-12-19,Keyword_1,"(hot, 74)"
4,2016-12-20,Keyword_1,"(hot, 76)"


In [116]:
tw_final2.rename(columns = {'variable':'rank'}, inplace = True)

In [117]:
# Seperates tuples in value
b = []
c = []

b.extend(a[1] for a in tw_final2['value'])
tw_final2['count'] = b

c.extend(a[0] for a in tw_final2['value'])
tw_final2['value'] = c

In [118]:
tw_final2.head()

,date,rank,value,count
0,2016-12-14,Keyword_1,fog,35
1,2016-12-15,Keyword_1,fog,31
2,2016-12-16,Keyword_1,hot,74
3,2016-12-19,Keyword_1,hot,74
4,2016-12-20,Keyword_1,hot,76


In [150]:
# Creates dataframe of total tweets for each day
df_count = dffinaltrained.groupby(['date']).count()

In [151]:
# Resets index
df_count.reset_index(level = 0, inplace = True)

In [152]:
df_count.rename(columns = {'text':'total_count'}, inplace = True)

In [153]:
df_count.head()

,date,total_count
0,2016-12-14,300
1,2016-12-15,255
2,2016-12-16,795
3,2016-12-19,791
4,2016-12-20,804


In [163]:
# Joins dataframes tw_final2 and df_count on date
merged = pd.merge(tw_final2, df_count, how='inner', on='date')

In [164]:
# Creates column for %daily word occurrence
merged['%daily word occurrence'] = (merged['count'] / merged['total_count']) * 100

In [165]:
merged.rename(columns = {'value':'Key_word'}, inplace = True)

In [166]:
merged.head()

,date,rank,Key_word,count,total_count,%daily word occurrence
0,2016-12-14,Keyword_1,fog,35,300,11.666667
1,2016-12-14,Keyword_2,hot,33,300,11.000000
2,2016-12-14,Keyword_3,cold,33,300,11.000000
3,2016-12-14,Keyword_4,sun,32,300,10.666667
4,2016-12-14,Keyword_5,warm,17,300,5.666667
